
```
States (Nodes):
Initial State (Start): The starting state before any calculation begins.
currentState = Start
a = 0
b = 1
counter = 0

while (currentState != End) {
    if (currentState == Start) {
        currentState = Calculate
    } else if (currentState == Calculate) {
        display a
        nextTerm = a + b
        if (counter >= n) {
            currentState = End
        } else {
            currentState = Update
        }
    } else if (currentState == Update) {
        a = b
        b = nextTerm
        counter = counter + 1
        currentState = Calculate
    }
}
```

In [8]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
    chain,
)
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from typing_extensions import TypedDict


class GraphState(TypedDict, total=False):
    a: int
    b: int
    next: int
    counter: int
    suite: list[int]


workflow = StateGraph(GraphState)

MAX = 5


@chain
def init(state: GraphState) -> GraphState:
    return {"a": 0, "b": 1, "counter": 0, "suite": []}


@chain
def calculate(state: GraphState) -> GraphState:
    next = state["a"] + state["b"]
    state["next"] = next
    state["suite"].append(next)
    return state


@chain
def is_end(state: GraphState) -> str:
    if state["counter"] < 5:
        return "update"
    else:
        return "finish"


@chain
def update(state: GraphState) -> GraphState:
    state["a"] = state["b"]
    state["b"] = state["next"]
    state["counter"] = state["counter"] + 1
    return state


# Define the nodes
workflow.add_node("init", init)
workflow.add_node("calculate", calculate)
workflow.add_node("update", update)
workflow.set_entry_point("init")
workflow.add_edge("init", "calculate")
workflow.add_conditional_edges(
    "calculate",
    is_end,
    {
        "update": "update",
        "finish": END,
    },
)
workflow.add_edge("update", "calculate")


app = workflow.compile()
app.invoke({})

{'a': 5, 'b': 8, 'next': 13, 'counter': 5, 'suite': [1, 2, 3, 5, 8, 13]}